### Reference 
Code: https://github.com/serengil/tensorflow-101/blob/master/python/deep-face-real-time.py   
Description: https://sefiks.com/2018/08/06/deep-face-recognition-with-keras/

Import

In [2]:
import numpy as np
import cv2

from keras.models import Model, Sequential
from keras.layers import Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Flatten, Dense, Dropout, Activation
from PIL import Image
from keras.preprocessing.image import load_img, save_img, img_to_array
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing import image
import matplotlib.pyplot as plt

from os import listdir

RGB color code for texts. 0, 255, 0 is lime

In [3]:
color = (0,255,0)

Create a classifier for face detections

In [4]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades+'haarcascade_frontalface_default.xml')

### Define functions

In [5]:
# Preprocess_input normalizes input in scale of [-1, +1]. You must apply same normalization in prediction.
# Devide the array by 127.5 and substract by 1
# Ref: https://github.com/keras-team/keras-applications/blob/master/keras_applications/imagenet_utils.py (Line 45)

def preprocess_image(image):
    img = img_to_array(image)
    img = np.expand_dims(img, axis=0)
    
    img = preprocess_input(img)
    return img

In [6]:
# Calculate a cosine distance between two values. Distance between similar vectors should be low.

def findCosineDistance(source_representation, test_representation):
    a = np.matmul(np.transpose(source_representation), test_representation)
    b = np.sum(np.multiply(source_representation, source_representation))
    c = np.sum(np.multiply(test_representation, test_representation))
    return 1 - (a / (np.sqrt(b) * np.sqrt(c)))

In [7]:
# Create a CNN 
# Download data from https://drive.google.com/file/d/1CPSeum3HpopfomUEK1gybeuIVoeJT_Eo/view?usp=sharing

def loadVggFaceModel():
    model = Sequential()
    model.add(ZeroPadding2D((1,1),input_shape=(224,224, 3)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(256, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(ZeroPadding2D((1,1)))
    model.add(Convolution2D(512, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2,2), strides=(2,2)))

    model.add(Convolution2D(4096, (7, 7), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(4096, (1, 1), activation='relu'))
    model.add(Dropout(0.5))
    model.add(Convolution2D(2622, (1, 1)))
    model.add(Flatten())
    model.add(Activation('softmax'))
    
    # Load pretrained weights for VGGFace
    from keras.models import model_from_json
    model.load_weights('/Users/jake/Desktop/CS490/Project/vgg_face_weights.h5')
    
    # Generate model with input and output
    vgg_face_descriptor = Model(inputs=model.layers[0].input, outputs=model.layers[-2].output)

    return vgg_face_descriptor  # return the model

### Real time face recognition

In [8]:
# Generate a model by function call
model = loadVggFaceModel()
model.summary()

2022-05-05 10:31:28.775658: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 zero_padding2d_input (Input  [(None, 224, 224, 3)]    0         
 Layer)                                                          
                                                                 
 zero_padding2d (ZeroPadding  (None, 226, 226, 3)      0         
 2D)                                                             
                                                                 
 conv2d (Conv2D)             (None, 224, 224, 64)      1792      
                                                                 
 zero_padding2d_1 (ZeroPaddi  (None, 226, 226, 64)     0         
 ng2D)                                                           
                                                                 
 conv2d_1 (Conv2D)           (None, 224, 224, 64)      36928     
                                                             

Use the cnn model and assign new images captured by OpenCV FR Dataset Generation.ipynb. We can say the images are train datasets.
The output from the cnn is saved in dictionary. 

In [9]:
# Initialize a file name and dictionary
new_pictures_folder = "New Faces"
new_pictures = dict()

# Iterate a folder ("New Faces") to folder ("User name") to user's 150 face images
# String beocomes file path by listdir
for folder_name in listdir(new_pictures_folder):
    if folder_name != '.DS_Store':  # for mac users
        
        folder_path = new_pictures_folder+'/'+folder_name  # create a path for the next folder to iterate
        
        pred = []  # initialize and reset a list
        
        for file_name in listdir(folder_path):
            file_name = folder_path+'/'+file_name  # create a path for the files
            face_img = load_img(file_name, target_size=(224, 224))  # load images by the path
            pred.append(model.predict(preprocess_image(face_img))[0,:])  # get values from each images by model.predict
            
        new_pictures[folder_name] = pred  # assign the list to dict

In [13]:
img = load_img("jackman.png", target_size=(224, 224))
img = preprocess_image(img)
value= model.predict(img)[0,:]
print(value)
print(len(value))

found = 0 

for k in new_pictures:  # k is a key of dict which is user's name
    folder = new_pictures[k]
    for i in folder:
        distance = findCosineDistance(i, value)
                    
        # If cosine distance is small, consider it is k (user's name)
        if(distance < 0.30): 
            print(k)
            found = 1
            break
        # if found image is not in user's face dataset, display unknown
    if(found == 0): 
        print('Unknown')

[ 4.685139  -0.9130396  0.7525303 ... -1.280415  -0.6542321  0.7728258]
2622
Unknown
Unknown
Unknown


The saved values in the dictionary is compared with images (we can say test datasets) extracted from a real time video.   
If a cosine distance between values in the dictionary and the values from webcam images is very small, it is considered as the same face and displays the user's name

In [3]:
# Open webcam
cap = cv2.VideoCapture(0)

while(True):
    ret, img = cap.read()  # get a image from webcam capture
    faces = face_cascade.detectMultiScale(img, 1.3, 5)  # find faces in images

    for (x,y,w,h) in faces:  # x, y represents initial posions in a graph. w is width and h is height
        if w > 130: 

            detected_face = img[int(y):int(y+h), int(x):int(x+w)] # crop detected face
            detected_face = cv2.resize(detected_face, (224, 224)) # resize to 224x224

            img_pixels = preprocess_image(detected_face)
            
            # Assign the image extracted from webcam
            captured_representation = model.predict(img_pixels)[0,:]

            found = 0  # counter
            
            for k in new_pictures:  # k is a key of dict which is user's name
                folder = new_pictures[k]
                for i in folder:
                    distance = findCosineDistance(i, captured_representation)
                    
                    # If cosine distance is small, consider it is k (user's name)
                    if(distance < 0.30): 
                        cv2.putText(img, k, (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
                    
                        found = 1
                        break

            # Connect face and text
            cv2.line(img,(int((x+x+w)/2),y+15),(x+w,y-20),color,1)
            cv2.line(img,(x+w,y-20),(x+w+10,y-20),color,1)
            
            # if found image is not in user's face dataset, display unknown
            if(found == 0): 
                cv2.putText(img, 'unknown', (int(x+w+15), int(y-12)), cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)

    cv2.imshow('img',img)

    if cv2.waitKey(1) & 0xFF == ord('q'): # press q to quit
        break

# Kill open cv things
cap.release()
cv2.destroyAllWindows()
        

NameError: name 'face_cascade' is not defined